In [1]:
import pya

from modified_ebeam_pdk import TECHLAYERS
from zeropdk.layout import (
    layout_box,
    layout_waveguide,
    layout_rectangle,
)

First, we will create a mask with our layout tools :

In [2]:
layout = pya.Layout()
dbu = layout.dbu = 0.001
TOP = layout.create_cell("TOP")

origin = pya.DPoint(0, 0)
ex = pya.DVector(1, 0)
ey = pya.DVector(0, 1)

# Parameters
dy = 1
dy_pp = 1
dx = 1
rib_width = 0.5
slab_width = 2*(dy + dy_pp)

# Layout waveguide
layout_waveguide(TOP, layout.layer(TECHLAYERS["Si"]), [-dx/2*ex, dx/2*ex], rib_width)
layout_waveguide(TOP, layout.layer(TECHLAYERS["Si slab"]), [-dx/2*ex, dx/2*ex], slab_width)

# Layout doping
layout_box(TOP, layout.layer(TECHLAYERS["Si N"]), -dx/2*ex, dx/2*ex + (dy + dy_pp)*ey, ex)
layout_box(TOP, layout.layer(TECHLAYERS["Si P"]), -dx/2*ex,  dx/2*ex - (dy + dy_pp)*ey, ex)

# Layout doped contacts
layout_box(TOP, layout.layer(TECHLAYERS["Si N++"]), -dx/2*ex + dy*ey, dx/2*ex + (dy + dy_pp)*ey, ex)
layout_box(TOP, layout.layer(TECHLAYERS["Si P++"]), -dx/2*ex - dy*ey, dx/2*ex - (dy + dy_pp)*ey, ex)

# Layout vias (boundary conditions for simulation)
layout_box(TOP, layout.layer(TECHLAYERS["VC"]), -dx/2*ex + (dy + dy_pp - 0.5)*ey, dx/2*ex + (dy + dy_pp)*ey, ex)
layout_box(TOP, layout.layer(TECHLAYERS["VC"]), -dx/2*ex - (dy + dy_pp - 0.5)*ey, dx/2*ex - (dy + dy_pp)*ey, ex)

# Cross-section simulation layer
sim_padding = 1
layout_rectangle(TOP, layout.layer(TECHLAYERS["sim_crossSection"]), 0*ex, 0.01, slab_width + sim_padding, ex)

print("Wrote to demo_output.gds")
layout.write("demo_output.gds")

Wrote to demo_output.gds


You can use the Desktop app on Adroit to look at your geometry :



In our technology files (modified_eveam_pdk.py which references a .lyp file), we have added arguments to the layers. Three types are defined, and each have associated parameters :

* Si (zmin, zmax) : the silicon layers, and their respective extension outside of the layout plane.
* doping (dopinglevel) : layers that dope the silicon and their level, respectively. Positive doping means P-type, and negative doping N-type.
* contact : vias connect the silicon to routing metals layers, and are used as boundary condition contacts in electrical simulations.

We can use the utility [libGDSII](https://github.com/HomerReid/libGDSII) to convert the mask we just made to a mesh for charge simulations

In [4]:
!/home/ELE559/libGDSII_install/bin/GDSIIConvert --GMSH demo_output.gds

Detecting metallization structures on layer   1: ... 1 polygons on layer 1
Detecting metallization structures on layer   2: ... 1 polygons on layer 2
Detecting metallization structures on layer  20: ... 1 polygons on layer 20
Detecting metallization structures on layer  22: ... 1 polygons on layer 22
Detecting metallization structures on layer  24: ... 1 polygons on layer 24
Detecting metallization structures on layer  26: ... 1 polygons on layer 26
Detecting metallization structures on layer  40: ... 2 polygons on layer 40
Detecting metallization structures on layer 1000: ... 1 polygons on layer 1000
Wrote 9 metallization polygons to demo_output.geo.
Thank you for your support.
